In [54]:
import configparser
import tweepy
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import time

In [55]:
# get tokens
config = configparser.ConfigParser()
config.read('configfile.ini')
api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']
# authenticate
auth = tweepy.OAuth2AppHandler(api_key, api_key_secret)

In [56]:
api = tweepy.API(auth, wait_on_rate_limit=True)

In [57]:
hashtag = '#LockwoodandCo'
#start_date = datetime(2023, 1, 1, 0,0,0, tzinfo=timezone.utc)  # Convert to offset-aware datetime
#end_date = datetime(2023, 1, 27, 23,59,59, tzinfo=timezone.utc)  # Convert to offset-aware datetime


In [61]:
working_df=pd.read_csv('working_df.csv')


In [90]:
def get_tweets_dataframe(working_df, tweets, hashtag):
    working_df=working_df.drop_duplicates()
    index = len(working_df)
    for tweet in tweets:
        print(index)   
        working_df.loc[index,'tweet_id']=tweet.id
        working_df.loc[index,'created_at']=tweet.created_at
        working_df.loc[index,'user']=tweet.user.screen_name
        working_df.loc[index,'full_text']=tweet.full_text
        working_df.loc[index,'favorite_count']=tweet.favorite_count
        working_df.loc[index,'retweet_count']=tweet.retweet_count
        working_df.loc[index,'hashtags']=hashtag
        index += 1
    working_df=working_df.drop_duplicates()    
    working_df.to_csv('tweets.csv', index=False)                
    return working_df

In [72]:
tweets = tweepy.Cursor(api.search_tweets, q=hashtag, tweet_mode='extended').items()

In [73]:
working_df = get_tweets_dataframe(working_df, tweets, hashtag)

0 1658770124684361728 2023-05-17 09:43:10+00:00 MessShower RT @hollowgirlhan: My thoughts on @netflix and @NetflixUK canceling #LockwoodandCo: 

They thought a show that didn’t sexualize teens would… 0 40 #LockwoodandCo
1 1658770086797213697 2023-05-17 09:43:01+00:00 Trissy_11 They started as colleagues but became a family #lockwoodandco showcased such a beautiful dynamic of friendships &amp; love &amp; now we will never get to see them learn, grow, fight together &amp; fight for each other. #LockwoodandCoSeason2 #TogetherForLockwoodandCo #PrimeForLockwoodandCo https://t.co/kjcLXzvPPZ 0 0 #LockwoodandCo
2 1658769973962067971 2023-05-17 09:42:34+00:00 AlinchenBee RT @xoutersxpace: Hey, @DisneyPlus this show would do really great in your platform
#LockwoodandCo #SaveLockwoodandCo https://t.co/Hf8hbGYD… 0 510 #LockwoodandCo
3 1658769893754392578 2023-05-17 09:42:15+00:00 siomha_wilson RT @shotfox35: me watching Netflix get ripped to shreds for cancelling #LockwoodAndCo https://t.co/acrP5c

In [91]:
hashtag = '#SaveLockwoodandCo'
tweets = tweepy.Cursor(api.search_tweets, q=hashtag, tweet_mode='extended').items()

In [92]:
working_df = get_tweets_dataframe(working_df, tweets, hashtag)
working_df.tail()

20352
20353
20354
20355
20356
20357
20358
20359
20360
20361
20362
20363


/home/biogear13/miniconda3/envs/minimal_ds/lib/python3.10/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


20364
20365
20366
20367
20368
20369
20370
20371
20372
20373
20374
20375
20376
20377
20378
20379
20380
20381
20382
20383
20384
20385
20386
20387
20388
20389
20390
20391
20392
20393
20394
20395
20396
20397
20398
20399
20400
20401
20402
20403
20404
20405
20406
20407
20408
20409
20410
20411
20412
20413
20414
20415
20416
20417
20418
20419
20420
20421
20422
20423
20424
20425
20426
20427
20428
20429
20430
20431
20432
20433
20434
20435
20436
20437
20438
20439
20440
20441
20442
20443
20444
20445
20446
20447
20448
20449
20450
20451
20452
20453
20454
20455
20456
20457
20458
20459
20460
20461
20462
20463
20464
20465
20466
20467
20468
20469
20470
20471
20472
20473
20474
20475
20476
20477
20478
20479
20480
20481
20482
20483
20484
20485
20486
20487
20488
20489
20490
20491
20492
20493
20494
20495
20496
20497
20498
20499
20500
20501
20502
20503
20504
20505
20506
20507
20508
20509
20510
20511
20512
20513
20514
20515
20516
20517
20518
20519
20520
20521
20522
20523
20524
20525
20526
20527
20528
20529
2053

/tmp/ipykernel_808/2751524740.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df.loc[index,'tweet_id']=tweet.id


20700
20701
20702
20703
20704
20705
20706
20707
20708
20709
20710
20711
20712
20713
20714
20715
20716
20717
20718
20719
20720
20721
20722
20723
20724
20725
20726
20727
20728
20729
20730
20731
20732
20733
20734
20735
20736
20737
20738
20739
20740
20741
20742
20743
20744
20745
20746
20747
20748
20749
20750
20751
20752
20753
20754
20755
20756
20757
20758
20759
20760
20761
20762
20763
20764
20765
20766
20767
20768
20769
20770
20771
20772
20773
20774
20775
20776
20777
20778
20779
20780
20781
20782
20783
20784
20785
20786
20787
20788
20789
20790
20791
20792
20793
20794
20795
20796
20797
20798
20799
20800
20801
20802
20803
20804
20805
20806
20807
20808
20809
20810
20811
20812
20813
20814
20815
20816
20817
20818
20819
20820
20821
20822
20823
20824
20825
20826
20827
20828
20829
20830
20831
20832
20833
20834
20835
20836
20837
20838
20839
20840
20841
20842
20843
20844
20845
20846
20847
20848
20849
20850
20851
20852
20853
20854
20855
20856
20857
20858
20859
20860
20861
20862
20863
20864
20865
2086

Rate limit reached. Sleeping for: 343


24444
24445
24446
24447
24448
24449
24450
24451
24452
24453
24454
24455
24456
24457
24458
24459
24460
24461
24462
24463
24464
24465
24466
24467
24468
24469
24470
24471
24472
24473
24474
24475
24476
24477
24478
24479
24480
24481
24482
24483
24484
24485
24486
24487
24488
24489
24490
24491
24492
24493
24494
24495
24496
24497
24498
24499
24500
24501
24502
24503
24504
24505
24506
24507
24508
24509
24510
24511
24512
24513
24514
24515
24516
24517
24518
24519
24520
24521
24522
24523
24524
24525
24526
24527
24528
24529
24530
24531
24532
24533
24534
24535
24536
24537
24538
24539
24540
24541
24542
24543
24544
24545
24546
24547
24548
24549
24550
24551
24552
24553
24554
24555
24556
24557
24558
24559
24560
24561
24562
24563
24564
24565
24566
24567
24568
24569
24570
24571
24572
24573
24574
24575
24576
24577
24578
24579
24580
24581
24582
24583
24584
24585
24586
24587
24588
24589
24590
24591
24592
24593
24594
24595
24596
24597
24598
24599
24600
24601
24602
24603
24604
24605
24606
24607
24608
24609
2461

Rate limit reached. Sleeping for: 635


29740
29741
29742
29743
29744
29745
29746
29747
29748
29749
29750
29751
29752
29753
29754
29755
29756
29757
29758
29759
29760
29761
29762
29763
29764
29765
29766
29767
29768
29769
29770
29771
29772
29773
29774
29775
29776
29777
29778
29779
29780
29781
29782
29783
29784
29785
29786
29787
29788
29789
29790
29791
29792
29793
29794
29795
29796
29797
29798
29799
29800
29801
29802
29803
29804
29805
29806
29807
29808
29809
29810
29811
29812
29813
29814
29815
29816
29817
29818
29819
29820
29821
29822
29823
29824
29825
29826
29827
29828
29829
29830
29831
29832
29833
29834
29835
29836
29837
29838
29839
29840
29841
29842
29843
29844
29845
29846
29847
29848
29849
29850
29851
29852
29853
29854
29855
29856
29857
29858
29859
29860
29861
29862
29863
29864
29865
29866
29867
29868
29869
29870
29871
29872
29873
29874
29875
29876
29877
29878
29879
29880
29881
29882
29883
29884
29885
29886
29887
29888
29889
29890
29891
29892
29893
29894
29895
29896
29897
29898
29899
29900
29901
29902
29903
29904
29905
2990

,tweet_id,created_at,user,full_text,favorite_count,retweet_count,hashtag,hashtags
32707,1658087250188541952,2023-05-15 12:29:40+00:00,Godsgirljune,RT @viv_winternight: My 2 (or more) cents for ...,0,7,NaN,#SaveLockwoodandCo
32708,1658087240432594946,2023-05-15 12:29:38+00:00,va_rosemarie,@savelockwoodnco You're welcome. I hope your s...,3,1,NaN,#SaveLockwoodandCo
32709,1658087219708428289,2023-05-15 12:29:33+00:00,toobookish4you,Let’s get it back!! \n#SaveLockwoodandCo #Brin...,9,3,NaN,#SaveLockwoodandCo
32710,1658087161546039296,2023-05-15 12:29:19+00:00,Godsgirljune,"RT @dublinfives: Please @primevideouk, please ...",0,3,NaN,#SaveLockwoodandCo
32711,1658087147553841152,2023-05-15 12:29:16+00:00,Godsgirljune,"RT @aDEPRACofficial: If you haven’t already, p...",0,4,NaN,#SaveLockwoodandCo


In [87]:
hashtag = '#TogetherForLockwoodandCo'
tweets = tweepy.Cursor(api.search_tweets, q=hashtag, tweet_mode='extended').items()
working_df = get_tweets_dataframe(working_df, tweets, hashtag)
working_df.tail()

Rate limit reached. Sleeping for: 178


20506
20507
20508
20509
20510
20511
20512
20513


/home/biogear13/miniconda3/envs/minimal_ds/lib/python3.10/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


20514
20515
20516
20517
20518
20519
20520
20521
20522
20523
20524
20525
20526
20527
20528
20529
20530
20531
20532
20533
20534
20535
20536
20537
20538
20539
20540
20541
20542
20543
20544
20545
20546
20547
20548
20549
20550
20551
20552
20553
20554
20555
20556
20557
20558
20559
20560
20561
20562
20563
20564
20565
20566
20567
20568
20569
20570
20571
20572
20573
20574
20575
20576
20577
20578
20579
20580
20581
20582
20583
20584
20585
20586
20587
20588
20589
20590
20591
20592
20593
20594
20595
20596
20597
20598
20599
20600
20601
20602
20603
20604
20605
20606
20607
20608
20609
20610
20611
20612
20613
20614
20615
20616
20617
20618
20619
20620
20621
20622
20623
20624
20625
20626
20627
20628
20629
20630
20631
20632
20633
20634
20635
20636
20637
20638
20639
20640
20641
20642
20643
20644
20645
20646
20647
20648
20649
20650
20651
20652
20653
20654
20655
20656
20657
20658
20659
20660
20661
20662
20663
20664
20665
20666
20667
20668
20669
20670
20671
20672
20673
20674
20675
20676
20677
20678
20679
2068

,tweet_id,created_at,user,full_text,favorite_count,retweet_count,hashtag,hashtags
25201,1658304994813308930,2023-05-16 02:54:55+00:00,NicoletteNway,Fight for #PrimeForLockwoodandCo\n#SaveLockwoo...,2,0,NaN,#PrimeForLockwoodandCo
25202,1658297523126272001,2023-05-16 02:25:13+00:00,lily_is_typing,#SaveLockwoodandCo #PrimeForLockwoodandCo http...,1,0,NaN,#PrimeForLockwoodandCo
25203,1658297337238929408,2023-05-16 02:24:29+00:00,weirdasfcklife,yesterday we trended for both hashtags! don’t ...,4,0,NaN,#PrimeForLockwoodandCo
25204,1658287986088321026,2023-05-16 01:47:20+00:00,siomha_wilson,#PrimeForLockwoodandCo,1,0,NaN,#PrimeForLockwoodandCo
25205,1658226283862523905,2023-05-15 21:42:09+00:00,kieronlander,I mean we need to tag Disney and BBCOne everyw...,4,0,NaN,#PrimeForLockwoodandCo


In [84]:
hashtag = '#PrimeForLockwoodandCo'
tweets = tweepy.Cursor(api.search_tweets, q=hashtag, tweet_mode='extended').items()

In [85]:
working_df = get_tweets_dataframe(working_df, tweets, hashtag)
working_df.tail()

Rate limit reached. Sleeping for: 74


20447
20448
20449
20450
20451
20452
20453
20454
20455
20456
20457
20458
20459
20460
20461
20462
20463
20464
20465
20466
20467
20468
20469
20470
20471
20472
20473
20474
20475
20476
20477
20478
20479
20480
20481
20482
20483
20484
20485
20486
20487
20488
20489
20490
20491
20492
20493
20494
20495
20496
20497
20498
20499
20500
20501
20502
20503
20504
20505
20506
20507
20508
20509
20510
20511
20512
20513
20514
20515
20516
20517
20518
20519
20520
20521
20522
20523
20524
20525
20526
20527
20528
20529
20530
20531
20532
20533
20534
20535
20536
20537
20538
20539
20540
20541
20542
20543
20544
20545
20546
20547
20548
20549
20550
20551
20552
20553
20554
20555
20556
20557
20558
20559
20560
20561
20562
20563
20564
20565
20566
20567
20568
20569
20570
20571
20572
20573
20574
20575
20576
20577
20578
20579
20580
20581
20582
20583
20584
20585
20586
20587
20588
20589
20590
20591
20592
20593
20594
20595
20596
20597
20598
20599
20600
20601
20602
20603
20604
20605
20606
20607
20608
20609
20610
20611
20612
2061

,tweet_id,created_at,user,full_text,favorite_count,retweet_count,hashtag,hashtags
25201,1658304994813308930,2023-05-16 02:54:55+00:00,NicoletteNway,Fight for #PrimeForLockwoodandCo\n#SaveLockwoo...,2,0,NaN,#PrimeForLockwoodandCo
25202,1658297523126272001,2023-05-16 02:25:13+00:00,lily_is_typing,#SaveLockwoodandCo #PrimeForLockwoodandCo http...,1,0,NaN,#PrimeForLockwoodandCo
25203,1658297337238929408,2023-05-16 02:24:29+00:00,weirdasfcklife,yesterday we trended for both hashtags! don’t ...,4,0,NaN,#PrimeForLockwoodandCo
25204,1658287986088321026,2023-05-16 01:47:20+00:00,siomha_wilson,#PrimeForLockwoodandCo,1,0,NaN,#PrimeForLockwoodandCo
25205,1658226283862523905,2023-05-15 21:42:09+00:00,kieronlander,I mean we need to tag Disney and BBCOne everyw...,4,0,NaN,#PrimeForLockwoodandCo


In [93]:
import hashlib
# Create a hash function to generate anonymized values
def anonymize(value):
    # Convert the value to a string and hash it using SHA256 algorithm
    hashed_value = hashlib.sha256(str(value).encode()).hexdigest()
    # Take the first 8 characters of the hash as the anonymized value
    anonymized_value = hashed_value[:8]
    return anonymized_value

In [94]:
user_df = pd.DataFrame(working_df['user'].unique(), columns=['user'])
user_df['anonymized_user'] = user_df['user'].apply(anonymize)
working_df = working_df.merge(user_df, on='user', how='left')


In [95]:
working_df.to_csv('tweets.csv', index=False)

In [98]:
len(working_df)

19702

In [99]:
len(working_df['created_at'].unique())

13550